# QUBO for shortest path

### For each net $p \in \{nets\}$ and for each rout $r \in \{p\}$ (to include forks):

-Constraint to set one node to 1 for the ending node: $(1-\sum_i y_{p,r,i} + \sum_{i\neq j} 2y_{p,r,i}y_{p,r,j})$

-Constraint to set other nodes to 0 or 2: $(\sum_{j}y_{p,r,j})(2-\sum_{j}y_{p,r,j})^2= \sum_{i} y_{p,r,i}-2\sum_{i\neq j}y_{p,r,i}y_{p,r,j}+6\sum_{i\neq j,i\neq k,j\neq j}y_{p,r,i}y_{p,r,j}y_{p,r,k}$. Adding ancilla variables to make it QUBO:
$\sum_i y_{p,r,i}-2\sum_{i\neq j}y_{p,r,i}y_{p,r,j}+6\sum_{i\neq j,i\neq k,j\neq k}[w_{p,r,i,j}y_{p,r,k}]+6\sum_{i\neq j,i\neq k,j\neq k}[y_{p,r,i}y_{p,r,j}-2w_{p,r,i,j}(y_{p,r,i}+y_{p,r,j})+3w_{p,r,i,j}]$

-Constraint to set one node to $n$ for the starting node: $(\sum_iy_{p,r,i}-n)^2=n^2-(2n-1)\sum_i y_{p,r,i}+\sum_{i\neq j} 2y_{p,r,i}y_{p,r,j}$

### Constraint on edges for all of the nets:

$\sum_{i \in \{edges\}}\sum_{p, q \in \{nets\} p \neq q} \sum_{r \in p, s \in q} y_{p,r,i}y_{q,s,i}$

-This will allow routs (r or s) from the same net/fork (p or q) to overlap.

# <span style="color:red"> Very important:<span>

-Minimum distance objective function:  $w\sum_i y_i$ 

if $w < 1$ is not there, solutions with the path of 0 length would be a viable solutions too. This is because the constraints only add a penalty or reward of 1/-1 and the objective function's penalty grows linearly with the routs length. Therefore, it is imperetive that we nulify the effect of this growth with a small weight. A weight too big wont do shit and a weight too small will make the adiabatic gap very tiny and inceases the error rate. 

There can be two solutions for this problem:

(A) Assign a small constant weight to it. This can be tricky as the size of the problem changes.

(B) In order to bring the effect of objective function to the level of other items one can use $w < 1/d$ where $d$ is the distance of the nodes in the distance matrix. 

In [ ]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import FixedEmbeddingComposite
import dwave_networkx as dnx
import hybrid
import dimod
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
import sys
from pathlib import Path
pp = str(Path('.').absolute().parent)
if pp not in sys.path:
    sys.path.append(pp)

In [ ]:
#import the necessary shit
from refactor.essentials import (
    RectGridGraph, SA, optimize_qannealer, # create_qubo,
    is_this_an_answer,
)
from qpr.quantum_utils import find_embedding_minorminer, get_all_min_energy
from qpr.notebook_utils import make_ax_grid

In [ ]:
#New QUBO for all of the nets
from itertools import combinations_with_replacement as cwr

# We now define netlists as a dictionary consisting of a set of keys (netstart) and values (list of net end)
def create_qubo(G, nets, params={'weight_objective': 1, 'weight_end': 1, 'weight_start': 1, 'weight_others': 1,'weight_and': 6,'weight_net_distinction': 1}):
    Q = {}
    for num_net, start_net in enumerate(nets.keys()):
        
        for num_rout, end_net in enumerate(nets[start_net]):
            
            n = len(nets[start_net])
            # these are 'y0', 'y1', etc
            for var1, var2 in cwr(G.np2qv.values(), 2):
                Q[(f'{var1}n{num_net}r{num_rout}', f'{var2}n{num_net}r{num_rout}')]=0
            
            Starting_node=[]
            end_nodes={}
            other_nodes={}
            for item_1 in nets[start_net]:
                end_nodes[item_1]=[]
            for item_1 in G.nodes:
                if item_1 not in nets[start_net] and item_1 != start_net:
                    other_nodes[item_1]=[]
            
            # iterate over numerical edge labels
            for edge_pair, num_lab in G.np2l.items():
                # objective
                w1=params['weight_objective']
                Q[(f'y{num_lab}n{num_net}r{num_rout}',f'y{num_lab}n{num_net}r{num_rout}')]=1*w1
                # According to Arash, VPR defines nets as having one start and multiple ends
                #Starting node
                if edge_pair[0] == start_net or edge_pair[1] == start_net:
                    Starting_node.append(num_lab)
                #end node
                if edge_pair[0] in nets[start_net]:
                    end_nodes[edge_pair[0]].append(num_lab)
                if edge_pair[1] in nets[start_net]:
                    end_nodes[edge_pair[1]].append(num_lab)
                #other nodes
                if edge_pair[0] in other_nodes:
                    other_nodes[edge_pair[0]].append(num_lab)
                if edge_pair[1] in other_nodes:
                    other_nodes[edge_pair[1]].append(num_lab)   
            
            #constraint on end nodes
            w2=params['weight_end']
            # each item is a list of (numerical) edge labels ending in the node
            for node, num_lab_list in end_nodes.items():
                # for each pair of edges
                for i,j in cwr(num_lab_list,2):
                    if i==j:
                        #######Removing end node edges from the objective
                        #Q[(f'y{i}',f'y{j}')]+=-1*w1 
                        ##############################
                        Q[(f'y{i}n{num_net}r{num_rout}',f'y{j}n{num_net}r{num_rout}')]+=-1*w2
                    else:
                        Q[(f'y{i}n{num_net}r{num_rout}',f'y{j}n{num_net}r{num_rout}')]+=2*w2
            
            #constraint on other nodes
            w3=params['weight_others']
            # yiyj and weightm
            w_and=params['weight_and']
            
            # iterate over numerical edge labels
        
            for node, num_lab_list in other_nodes.items():
                for i,j in cwr(num_lab_list,2):
                    if i==j:
                        Q[(f'y{i}n{num_net}r{num_rout}',f'y{j}n{num_net}r{num_rout}')]+=w3
                    else:
                        Q[(f'y{i}n{num_net}r{num_rout}',f'y{j}n{num_net}r{num_rout}')]+=-2*w3 #2
                for i,j,k in cwr(num_lab_list,3):
                    if i !=j and j!=k and i !=k:
                        if (f'w{i}{j}n{num_net}r{num_rout}',f'y{k}n{num_net}r{num_rout}') not in Q:
                            Q[(f'w{i}{j}n{num_net}r{num_rout}',f'y{k}n{num_net}r{num_rout}')]=0
                        Q[(f'w{i}{j}n{num_net}r{num_rout}',f'y{k}n{num_net}r{num_rout}')]+=6*w3 #2
                        Q[(f'y{i}n{num_net}r{num_rout}',f'y{j}n{num_net}r{num_rout}')]+=6*w_and #2
                        if (f'w{i}{j}n{num_net}r{num_rout}',f'y{i}n{num_net}r{num_rout}') not in Q:
                            Q[(f'w{i}{j}n{num_net}r{num_rout}',f'y{i}n{num_net}r{num_rout}')]=0
                        Q[(f'w{i}{j}n{num_net}r{num_rout}',f'y{i}n{num_net}r{num_rout}')]+=-12*w_and
                        if (f'w{i}{j}n{num_net}r{num_rout}',f'y{j}n{num_net}r{num_rout}') not in Q:
                            Q[(f'w{i}{j}n{num_net}r{num_rout}',f'y{j}n{num_net}r{num_rout}')]=0
                        Q[(f'w{i}{j}n{num_net}r{num_rout}',f'y{j}n{num_net}r{num_rout}')]+=-12*w_and
                        if (f'w{i}{j}n{num_net}r{num_rout}',f'w{i}{j}n{num_net}r{num_rout}') not in Q:
                            Q[(f'w{i}{j}n{num_net}r{num_rout}',f'w{i}{j}n{num_net}r{num_rout}')]=0
                        Q[(f'w{i}{j}n{num_net}r{num_rout}',f'w{i}{j}n{num_net}r{num_rout}')]+=18*w_and
        
                        
            #constraint on the starting node
            w4=params['weight_start']
            # Starting_node is a list of numerical labels and not a dict
            for i,j in cwr(Starting_node,2):
                if i==j:
                    #######Removing starting node edges from the objective
                    #Q[(f'y{i}',f'y{j}')]+=-1*w1 
                    ##############################
                    Q[(f'y{i}n{num_net}r{num_rout}',f'y{j}n{num_net}r{num_rout}')]+=-(2*n-1)*w4
                else:
                    Q[(f'y{i}n{num_net}r{num_rout}',f'y{j}n{num_net}r{num_rout}')]+=2*w4
    #Constrains on nets and routs
    w5=params['weight_net_distinction']
    
    #for num_net1, num_net2 in cwr(range(len(nets)),2):

    for num_net1, start_net1 in enumerate(nets.keys()):
        for num_net2, start_net2 in enumerate(nets.keys()):
            for num_rout1, _ in enumerate(nets[start_net1]):
                for num_rout2, _ in enumerate(nets[start_net2]):
                    for var in G.np2qv.values():
                        if num_net1 != num_net2:
                            Q[(f'{var}n{num_net1}r{num_rout1}', f'{var}n{num_net2}r{num_rout2}')] = w5
    return Q

In [ ]:
#prepare the netlist

nets={(0,0):[(2,1)], (0,1):[(1,1), (2,2)]}


G = RectGridGraph(3, 3)


In [ ]:
import pickle
params = {'weight_objective': 0.15,
 'weight_end': 0.99,
 'weight_start': 0.82,
 'weight_others': 0.59,
 'weight_and': 1.7,
 'weight_net_distinction': 1}
Q=create_qubo(G, nets, params)
#with open('Q10by10.pickle', 'wb') as f:
#    pickle.dump(Q, f, protocol=pickle.HIGHEST_PROTOCOL)
Q

In [ ]:
from dwave_qbsolv import QBSolv
response = QBSolv().sample_qubo(Q)
print("energies=" + str(list(response.data_vectors['energy'])))

In [ ]:
%%time
dwave_sampler = DWaveSampler(solver={'lower_noise': True, 'qpu': True})
A = dwave_sampler.edgelist
embedding, chain_len = find_embedding_minorminer(Q, A, num_tries=100)
## the shortest chain_len I've seen with num_tries=1000 is 5
## (SP: takes 2.5 mins on my machine, SAS: 1:08 on mine)
display(chain_len)

In [ ]:
connectivity_structure = dnx.chimera_graph(16,16)
fig=plt.figure(figsize=(25, 25))
dnx.draw_chimera_embedding(connectivity_structure, embedding)

# QPU

In [ ]:
fixed_sampler = FixedEmbeddingComposite(
            DWaveSampler(solver={'lower_noise': True, 'qpu': True}), embedding
            )
q_response = optimize_qannealer(fixed_sampler, Q, params={'chain_strength': 15, 'annealing_time': 100, 'num_reads': 9900, 'anneal_schedule': None})
display(q_response.first)
best_q_answer = q_response.first.sample

In [ ]:
G.draw()

edge_set = G.qubo_answer2node_pairs(q_response.samples()[0])
G.highlight_edge_list(edge_set)

In [ ]:
is_this_an_answer(q_response.samples()[0], G, net_start, net_end)

In [ ]:
def make_ax_grid(n, ax_h=4, ax_w=6, ncols=4):
    nrows = int(np.ceil(n / ncols))
    fig_h = nrows * ax_h
    fig_w = ncols * ax_w
    return plt.subplots(nrows=nrows, ncols=ncols, figsize=(fig_w, fig_h))

# Exact solver

In [ ]:
%%time
exact_response = dimod.ExactSolver().sample_qubo(Q)
display(exact_response.record)

In [ ]:
# .data() sorts by energy by defaults but returns an iterator (not a SampleSet)
# the iterator yields a named tuple
# .samples(n) sort by energy, take at most n samples, return a SampleArray
# which is a view, mapping the var names to the values (i.e returns dicts), It is
# indexable i.e. .samples()[10] works
# .record returns record array of Sample objects which is basically a 
# numpy-scliceable list of named tuples (samples). Also .record.energy
# returns a numpy array of energies, .record.samples returns a 2d numpy
# array of qubo answers etc.
# Iterating over the SampleSet, calls .samples() internally, i.e. it gets sorted
# .first calls data() internally so it does the sorting anyway!

# This function returns all the min energy solutions as a list of {var name: val} dicts
def get_all_min_energy(sample_set):
    min_energy = np.min(sample_set.record.energy)
    # use .record since it is slicing friendly, this returns a 2d-array-like recarray
    records = sample_set.record[sample_set.record.energy == min_energy]
    # make dicts out of each answer using the original var names (i.e. sample_set.variables)
    return [dict(zip(sample_set.variables, i.sample)) for i in records], min_energy

In [ ]:
def plot_all_exact_solutions(min_energy_sols):
    fig, axes = make_ax_grid(len(min_energy_sols))
    display(len(min_energy_sols))
    
    for ax, answer_dict in zip(axes.flat, min_energy_sols):
        G.draw(edge_labs=False, ax=ax)  # edge_labs=False)
    
        edge_set = G.qubo_answer2node_pairs(answer_dict)
        G.highlight_edge_list(edge_set, ax=ax)
    fig.tight_layout()

In [ ]:
min_energy_sols, _ = get_all_min_energy(exact_response)
plot_all_exact_solutions(min_energy_sols)

In [ ]:
def check_against_exact(ans,exact_min_energy_sols):
    #ans is the answer from QPU or hybrid solver. 
    #exact_min_energy_sols is the set of all possible solutions from the exact solver
    return (ans in exact_min_energy_sols)

In [ ]:
print(check_against_exact(q_response.samples()[0],min_energy_sols))

# hybrid solution

In [ ]:
# Construct a problem
offset=0.0
#vartype = dimod.BINARY
bqm = dimod.BinaryQuadraticModel.from_qubo(Q, offset)

# Define the workflow
iteration = hybrid.RacingBranches(
    hybrid.InterruptableTabuSampler(),
    hybrid.EnergyImpactDecomposer(size=2)
    | hybrid.QPUSubproblemAutoEmbeddingSampler()
    | hybrid.SplatComposer()
) | hybrid.ArgMin()
workflow = hybrid.LoopUntilNoImprovement(iteration, convergence=3)

# Solve the problem
init_state = hybrid.State.from_problem(bqm)
final_state = workflow.run(init_state).result()

# Print results
print("Solution: sample={.samples.first}".format(final_state))

In [ ]:
G.draw(edge_labs=False)  # edge_labs=False)

edge_set = G.qubo_answer2node_pairs(final_state.samples.first[0])
G.highlight_edge_list(edge_set)


# simulated  annealing

In order to optimize the parameters of QUBO (and later QPU) we employ s simulated annealing algorithm. 

### Best Answer so far:
-It seems so far as the chain strength is the most contrinuting factor!

In [ ]:
a=SA(2, params={'weight_objective': [0.58, 0.58, 0.58, 0, 0.1], 'weight_end': [1.05, 0, 2, 0, 0.1],
                                           'weight_start': [1.12, 0, 2, 0, 0.1] ,'weight_others': [0.55, 0, 2, 0, 0.1],
                                           'weight_and': [1.949, 0, 2, 0, 0.1],
                                           'chain_strength': [15, 15, 15, 0, 0.5], 'annealing_time': None, 
                                           'anneal_schedule':[[[0,0], [10,0.4], [90, 0.8], [100, 1]],[[[0,0],[0,0]], [[5, 30], [0.2,0.6]], [[10, 95], [0.6,0.9]], [[100, 100], [1, 1]]],[5, 0.05]] 
                                          },
                 T=1, T_min=0.4, alpha=0.8, max_iter=50)

In [ ]:
a.param_generator()
a.sol_

In [ ]:
a.costs

In [ ]:
a.sols

We cool down slower to search the space around the current answer better

In [ ]:
#a.T_min = 0.01
#a.alpha = 0.9
a.anneal()

In [ ]:
a.costs

In [ ]:
a.sols